In [ ]:
#default_exp learner

In [ ]:
#export
from fastai2.basics import *
from faststyle import *

In [ ]:
#export
@delegates(Learner.__init__)
def style_learner(dls, style_fns, loss_func=None, arch=None, **kwargs):
  arch = arch or transformer_net()
  return Learner(dls, arch, loss_func=loss_func, metrics=LossMetrics(loss_func.metric_names),**kwargs)

## Export -

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.ipynb.
Converted 02_loss.ipynb.
Converted 03_models.ipynb.
Converted 04_learner.ipynb.
Converted 05_callback.ipynb.
